In [1]:
import coinbasepro as cb
import numpy as np
import pandas as pd
import datetime as dt
import random

In [2]:
all_products = cb.PublicClient().get_products()

In [3]:
# View Sample Object to See Parameters
all_products[:1]

[{'id': 'AVAX-EUR',
  'base_currency': 'AVAX',
  'quote_currency': 'EUR',
  'quote_increment': Decimal('0.01'),
  'base_increment': Decimal('0.001'),
  'display_name': 'AVAX-EUR',
  'min_market_funds': Decimal('0.84'),
  'margin_enabled': False,
  'post_only': False,
  'limit_only': False,
  'cancel_only': False,
  'status': 'online',
  'status_message': '',
  'trading_disabled': False,
  'fx_stablecoin': False,
  'max_slippage_percentage': Decimal('0.03000000'),
  'auction_mode': False,
  'high_bid_limit_percentage': ''}]

In [4]:
# Only include coins with an ID that converts to USD
assets = [prod for prod in all_products if prod["quote_currency"] == "USD"]
num_unique_assets = 15

def output_rand_list(full_list, num_picks):
    output = []
    while len(output) < num_picks:
        new_choice = random.choice(full_list)
        if new_choice not in output:
            output.append(new_choice)
    return output
rand_output = output_rand_list(assets, num_unique_assets)
rand_assets = [asset['base_currency'] for asset in rand_output]
prod_ids = [asset['id'] for asset in rand_output]

# Number of digits to round each crypto spot price
round_increment = [len(str(asset['quote_increment'])) - 2 for asset in rand_output]
round_increment

[2, 4, 3, 5, 2, 4, 2, 3, 4, 3, 2, 4, 3, 3, 4]

In [5]:
# Generate random dates for sample data output
num_transactions = 2000

def generate_dates(iterations):
    datestrings = []
    years = np.arange(2019, 2023)
    months = np.arange(1, 13)
    days = np.arange(1, 32)
    hours = np.arange(24)
    min_secs = np.arange(60)
    thirties = [4,6,9,11] # months with 30 days
    thirty_one = [1,3,5,7,8,10,12] # months with 31 days
    for _ in range(iterations):
        year = random.choice(years)
        month = random.choice(months)
        if month in thirties:
            day = random.choice(days[:-1])
        elif month in thirty_one:
            day = random.choice(days)
        else:
            day = int(random.choice(days[:-3])) # february
        hour = random.choice(hours)
        minute = random.choice(min_secs)
        second = random.choice(min_secs)
        date = dt.datetime(year, month, day, hour, minute, second)
        datestrings.append(date.isoformat(timespec='seconds') +"Z")
    return datestrings
    
sample_dates = generate_dates(num_transactions)
sample_dates[:3]

['2021-08-31T10:48:32Z', '2022-03-04T21:37:41Z', '2021-09-29T20:22:53Z']

In [6]:
# Generate sample rate ranges using the last 300 days worth of historic price data for each coin
rate_ranges = {}
pc = cb.PublicClient()
for i in range(len(prod_ids)):
    rates_300 = pc.get_product_historic_rates(product_id=prod_ids[i], granularity=86400)
    rate_ranges[rand_assets[i]] = {"min_rate": float(min([x["low"] for x in rates_300])),
                                  "max_rate": float(max([x["high"] for x in rates_300])),
                                  "round_rate": round_increment[i],
                                  "current_amount": 0}
rate_ranges.keys()

dict_keys(['AVAX', 'SEAM', 'NEON', 'HBAR', 'FARM', 'RLC', 'SHIB', 'HNT', 'VOXEL', 'AURORA', 'DASH', 'TRAC', 'ALICE', 'UMA', 'GFI'])

In [7]:
# Build out the transactions for the sample data
all_transactions = []
sample_dates.sort()
transaction_types = ["Buy", "Recieve", "Sell", "Convert"]
current_transactions = 0
assets = list(rate_ranges.keys())

def get_transactions(asset, transaction, tran_amount, date):
    sample_transaction = {}
    price = random.uniform(rate_ranges[asset]["min_rate"], rate_ranges[asset]["max_rate"])
    if (transaction == "Buy") | (transaction == "Recieve"):
        rate_ranges[asset]["current_amount"] +=  tran_amount
        fees = 0
        total =round(price * tran_amount, 2)
        if transaction == "Buy":
            notes = f"Bought {tran_amount} {asset} for ${total} USD"
        else:
            notes = f"Recieved {tran_amount} {asset} from an external source"
    else:
        rate_ranges[asset]["current_amount"] -= tran_amount
        fees = (price * tran_amount) * 0.015
        total = round((price * tran_amount) + fees, 2)
        if transaction == "Sell":
            notes = f"Sold {tran_amount} {asset} for ${total} USD"
        else:
            remaining = assets.copy()
            remaining.remove(asset)
            new_asset = random.choice(remaining)
            new_quantity = (tran_amount*price) / random.uniform(rate_ranges[new_asset]["min_rate"], rate_ranges[new_asset]["max_rate"])
            rate_ranges[new_asset]["current_amount"] += new_quantity
            notes = f"Converted {tran_amount} {asset} to {round(new_quantity, 2)} {new_asset}"
        
    sample_transaction["Timestamp"] = date
    sample_transaction["Transaction Type"] = transaction
    sample_transaction["Asset"] = asset
    sample_transaction["Quantity Transacted"] = tran_amount
    sample_transaction["Spot Price Currency"] = "USD"
    sample_transaction["Spot Price at Transaction"] = round(price, rate_ranges[asset]["round_rate"])
    sample_transaction["Subtotal"] = round(price * tran_amount, 2)
    sample_transaction["Total (inclusive of fees and/or spread)"] = total
    sample_transaction["Fees and/or Spread"] = round(fees, 2)
    sample_transaction["Notes"] = notes
    return sample_transaction
    
        
while current_transactions < num_transactions:
    asset = random.choice(assets)
    transaction = random.choice(transaction_types)
    tran_amount = random.randrange(0,100)
    if ((transaction == "Sell") | (transaction == "Convert")) & (rate_ranges[asset]["current_amount"] < tran_amount):
        transaction = random.choice(["Buy", "Recieve"])
    all_transactions.append(get_transactions(asset, transaction, tran_amount, sample_dates[current_transactions]))
    current_transactions += 1
    

In [8]:
sample_data = pd.DataFrame(all_transactions)
sample_data

,Timestamp,Transaction Type,Asset,Quantity Transacted,Spot Price Currency,Spot Price at Transaction,Subtotal,Total (inclusive of fees and/or spread),Fees and/or Spread,Notes
0,2019-01-02T22:35:35Z,Recieve,HNT,74,USD,6.9710,515.87,515.87,0.00,Recieved 74 HNT from an external source
1,2019-01-03T14:24:41Z,Buy,SHIB,3,USD,0.0000,0.00,0.00,0.00,Bought 3 SHIB for $0.0 USD
2,2019-01-03T17:44:59Z,Recieve,GFI,35,USD,3.4539,120.89,120.89,0.00,Recieved 35 GFI from an external source
3,2019-01-04T06:07:25Z,Buy,NEON,60,USD,1.0260,61.54,61.54,0.00,Bought 60 NEON for $61.54 USD
4,2019-01-05T00:18:04Z,Recieve,GFI,64,USD,0.5887,37.68,37.68,0.00,Recieved 64 GFI from an external source
...,...,...,...,...,...,...,...,...,...,...
1995,2022-12-29T10:20:24Z,Recieve,VOXEL,73,USD,0.3577,26.11,26.11,0.00,Recieved 73 VOXEL from an external source
1996,2022-12-30T05:30:54Z,Convert,TRAC,77,USD,1.1307,87.06,88.37,1.31,Converted 77 TRAC to 668.15 HBAR
1997,2022-12-30T20:36:50Z,Recieve,SEAM,77,USD,12.3763,952.98,952.98,0.00,Recieved 77 SEAM from an external source
1998,2022-12-31T20:27:07Z,Recieve,FARM,25,USD,94.2800,2357.05,2357.05,0.00,Recieved 25 FARM from an external source


In [10]:
sample_data.to_csv("Resources/sample_data.csv")